In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import numpy as np

# defining model architecture 
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16 * 16 * 16)
        x = self.fc1(x)
        return x

#  settingup hyperparameters
batch_size = 32
learning_rate = 0.001
epochs = 50
patience = 5  # Early stopping patience

# preparing data loaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# initializing model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# training loop with early stopping
best_val_loss = np.inf
epochs_no_improve = 0
for epoch in range(epochs):
    model.train()
    train_loss, train_accuracy = 0.0, 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        train_accuracy += (predicted == labels).sum().item()

    train_loss /= len(train_loader.dataset)
    train_accuracy /= len(train_loader.dataset)
    
    # validation phase
    model.eval()
    val_loss, val_accuracy = 0.0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            val_accuracy += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy /= len(val_loader.dataset)
    
    print(f"Epoch {epoch+1}/{epochs}, "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    
    # checking early stopping 
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping")
            break

Files already downloaded and verified
Epoch 1/50, Train Loss: 1.4839, Train Acc: 0.4758, Val Loss: 1.2886, Val Acc: 0.5530
Epoch 2/50, Train Loss: 1.1971, Train Acc: 0.5829, Val Loss: 1.1932, Val Acc: 0.5833
Epoch 3/50, Train Loss: 1.0843, Train Acc: 0.6262, Val Loss: 1.1460, Val Acc: 0.6013
Epoch 4/50, Train Loss: 1.0209, Train Acc: 0.6502, Val Loss: 1.1145, Val Acc: 0.6202
Epoch 5/50, Train Loss: 0.9712, Train Acc: 0.6656, Val Loss: 1.0991, Val Acc: 0.6265
Epoch 6/50, Train Loss: 0.9374, Train Acc: 0.6774, Val Loss: 1.1183, Val Acc: 0.6253
Epoch 7/50, Train Loss: 0.9084, Train Acc: 0.6867, Val Loss: 1.0969, Val Acc: 0.6253
Epoch 8/50, Train Loss: 0.8845, Train Acc: 0.6936, Val Loss: 1.1274, Val Acc: 0.6151
Epoch 9/50, Train Loss: 0.8616, Train Acc: 0.7034, Val Loss: 1.1255, Val Acc: 0.6218
Epoch 10/50, Train Loss: 0.8478, Train Acc: 0.7076, Val Loss: 1.1352, Val Acc: 0.6115
Epoch 11/50, Train Loss: 0.8290, Train Acc: 0.7129, Val Loss: 1.1124, Val Acc: 0.6275
Epoch 12/50, Train Loss: 